In [6]:
# импорт библиотек
import pandas as pd
!pip install pubchempy
import time
import numpy as np


#**Извлечение SMILES**

In [7]:
# загрузка датасетов
!gdown 1-8Seu7jUGobRTtM6N-38hPhToT0_eB8A
!unzip -q /content/data.zip
train_dataset=pd.read_csv('/content/train_set.csv')
test_dataset=pd.read_csv('/content/test_set.csv')
#просто нажимаем Enter

Downloading...
From: https://drive.google.com/uc?id=1-8Seu7jUGobRTtM6N-38hPhToT0_eB8A
To: /content/data.zip
100% 11.9k/11.9k [00:00<00:00, 27.2MB/s]


In [ ]:
train_dataset

,Pubchem_ID,taste_cluster
0,95609,0
1,104224,0
2,5284499,6
3,10886,4
4,12978217,3
...,...,...
2059,229385,1
2060,62387,2
2061,8878,7
2062,61262,1


Мы видим, что здесь есть выбросы в колонке `Pubchem_ID` - некоторые значения отрицательные. Для ID такого быть не может. Уберём эти строки из тренировочного датасета

In [ ]:
train_dataset_filter = train_dataset[train_dataset['Pubchem_ID']>0]

In [ ]:
train_dataset_filter

,Pubchem_ID,taste_cluster
0,95609,0
1,104224,0
2,5284499,6
3,10886,4
4,12978217,3
...,...,...
2059,229385,1
2060,62387,2
2061,8878,7
2062,61262,1


Теперь можем извлекать SMILES. Сделаем это через библиотеку pubchempy

In [ ]:
import pubchempy as pcp
def get_canonical_smiles(pubchem_id: str):
    try:
        compound = pcp.Compound.from_cid(int(pubchem_id))
        return compound.canonical_smiles
    except:
        return None

In [ ]:
train_smiles = []
for pubchem_id in train_dataset_filter['Pubchem_ID']:
    smiles = get_canonical_smiles(str(pubchem_id))
    train_smiles.append(smiles)
    time.sleep(0.1)
train_dataset_filter['canonical_SMILES'] = train_smiles

/tmp/ipython-input-3929576659.py:5: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  return compound.canonical_smiles


In [ ]:
train_dataset_filter

,Pubchem_ID,taste_cluster,canonical_SMILES
0,95609,0,CCC(C(=O)CC)O
1,104224,0,CCCCCCCCCCC(OC)OC
2,5284499,6,CCCCCCCCC=CCCCCCCCCO
3,10886,4,CCCCCC(=O)OCCCCC
4,12978217,3,CC=CC(=O)C1C(C=CCC1(C)C)C
...,...,...,...
2059,229385,1,CCCCCCCCCCOC(=O)CC
2060,62387,2,CC1(C2CCC1(C(C2)OC=O)C)C
2061,8878,7,CCCCCCC(=O)OCC=C
2062,61262,1,CC(=O)OCC1=CCC2CC1C2(C)C


In [ ]:
train_dataset_filter.to_csv('train_SMILES.csv', index=False)

In [ ]:
train_dataset_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Pubchem_ID        2064 non-null   int64 
 1   taste_cluster     2064 non-null   int64 
 2   canonical_SMILES  2064 non-null   object
dtypes: int64(2), object(1)
memory usage: 48.5+ KB


Как мы видим, пропуски отсутствуют, значит все остальные ID были корректными. В итоге после очистки датасета у нас осталось 99% данных, то есть потерь от некорректных ID практически не произошло. То же самое сделаем для `test_dataset`

In [ ]:
test_dataset

,Pubchem_ID
0,32594
1,7130
2,7455
3,31226
4,17000
...,...
883,26334
884,104421
885,21263168
886,5368236


Посмотрим также на наличие выбросов

In [ ]:
test_dataset_filter=test_dataset[test_dataset['Pubchem_ID']>0]
test_dataset_filter

,Pubchem_ID
0,32594
1,7130
2,7455
3,31226
4,17000
...,...
883,26334
884,104421
885,21263168
886,5368236


Мы видим, что здесь тоже были выбросы - 5 отрицательных ID. Извлечём SMILES через pubchempy для оставшихся ID

In [ ]:
test_smiles = []
for pubchem_id in test_dataset_filter['Pubchem_ID']:
    smiles = get_canonical_smiles(str(pubchem_id))
    test_smiles.append(smiles)
    time.sleep(0.1)
test_dataset_filter['canonical_SMILES'] = test_smiles

/tmp/ipython-input-3929576659.py:5: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  return compound.canonical_smiles


In [ ]:
test_dataset_filter

,Pubchem_ID,canonical_SMILES
0,32594,CC(C)CC1=NC=CN=C1OC
1,7130,CC(C)CC1=NC2=CC=CC=C2C=C1
2,7455,CC1=CC=C(C=C1)C(=O)OC
3,31226,CC(=O)OCCCC1=CC=CC=C1
4,17000,CC1CCCC(C1=O)(C)C
...,...,...
883,26334,CCC1=NC=C(N=C1C)C
884,104421,CCOC1=NC(=CN=C1)C
885,21263168,CCCCCCCCCCCC(=O)OCCC(C)CCC=C(C)C
886,5368236,CCCC=CCOC(=O)CC1=CC=CC=C1


In [ ]:
test_dataset_filter.to_csv('test_SMILES.csv', index=False)

In [ ]:
test_dataset_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Pubchem_ID        888 non-null    int64 
 1   canonical_SMILES  888 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.0+ KB


Здесь пропуски также отсутсвуют. Теперь можно переходить к генерации дескрипторов

In [ ]:
train_SMILES=pd.read_csv('/content/train_SMILES.csv')
test_SMILES=pd.read_csv('/content/test_SMILES.csv')

#**Генерация Mordred-дескрипторов**

In [3]:
!pip install mordred
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 53.8 MB/s eta 0:00:00


In [10]:
from mordred import Calculator, descriptors
from rdkit import Chem

In [11]:
train_df_optimal=pd.read_csv('/content/train_SMILES.csv')
test_df_optimal=pd.read_csv('/content/test_SMILES.csv')

In [12]:
calc = Calculator(descriptors)

results = []
train_smiles_list=train_df_optimal['canonical_SMILES'].tolist()
for smiles in train_smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    desc = calc(mol)
    desc_dict = desc.asdict()
    desc_dict['SMILES'] = smiles
    results.append(desc_dict)

# Создаем DataFrame
train_df_mordred = pd.DataFrame(results)


In [13]:
train_df_mordred

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,SMILES
0,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,9.331737,2.095294,4.190588,9.331737,1.166467,2.900838,...,34.513363,116.083730,5.804186,68,8,30.0,31.0,4.722222,2.111111,CCC(C(=O)CC)O
1,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,18.240791,2.057476,4.114953,18.240791,1.216053,3.524406,...,43.814886,216.208930,5.028115,520,14,56.0,56.0,5.861111,4.000000,CCCCCCCCCCC(OC)OC
2,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,23.412409,1.975377,3.950753,23.412409,1.232232,3.757227,...,48.426412,268.276616,4.877757,1140,16,70.0,68.0,6.250000,5.000000,CCCCCCCCC=CCCCCCCCCO
3,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,15.505483,2.045700,4.091400,15.505483,1.192729,3.379798,...,40.947839,186.161980,5.318914,334,11,48.0,47.0,5.361111,3.416667,CCCCCC(=O)OCCCCC
4,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,15.961560,2.420953,4.841907,15.961560,1.140111,3.534439,...,45.774443,192.151415,5.651512,285,21,68.0,78.0,6.645833,3.180556,CC=CC(=O)C1C(C=CCC1(C)C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,18.169374,2.017464,4.034927,18.169374,1.211292,3.524117,...,43.650894,214.193280,5.224226,538,13,56.0,55.0,5.861111,3.916667,CCCCCCCCCCOC(=O)CC
2060,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,16.040654,2.628414,4.984756,16.040654,1.233896,3.532886,...,62.626381,182.130680,5.875183,215,22,74.0,94.0,5.597222,2.770833,CC1(C2CCC1(C(C2)OC=O)C)C
2061,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,14.445702,2.039694,4.079389,14.445702,1.203809,3.298990,...,39.535875,170.130680,5.671023,262,10,44.0,43.0,5.111111,3.166667,CCCCCCC(=O)OCC=C
2062,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,17.137765,2.563547,5.127093,17.137765,1.224126,3.597390,...,47.370948,194.130680,6.066584,305,18,76.0,91.0,5.756944,2.944444,CC(=O)OCC1=CCC2CC1C2(C)C


In [14]:
non_numeric_cols = train_df_mordred.select_dtypes(exclude=[np.number]).columns
non_numeric_cols = [col for col in non_numeric_cols if col != 'SMILES']
train_df_with_smiles = train_df_mordred.drop(columns=non_numeric_cols)

In [15]:
train_df_with_smiles

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,SMILES
0,0,0,9.331737,2.095294,4.190588,9.331737,1.166467,2.900838,2.595082,0.324385,...,34.513363,116.083730,5.804186,68,8,30.0,31.0,4.722222,2.111111,CCC(C(=O)CC)O
1,0,0,18.240791,2.057476,4.114953,18.240791,1.216053,3.524406,3.204505,0.213634,...,43.814886,216.208930,5.028115,520,14,56.0,56.0,5.861111,4.000000,CCCCCCCCCCC(OC)OC
2,0,0,23.412409,1.975377,3.950753,23.412409,1.232232,3.757227,4.018055,0.211477,...,48.426412,268.276616,4.877757,1140,16,70.0,68.0,6.250000,5.000000,CCCCCCCCC=CCCCCCCCCO
3,0,0,15.505483,2.045700,4.091400,15.505483,1.192729,3.379798,3.152585,0.242507,...,40.947839,186.161980,5.318914,334,11,48.0,47.0,5.361111,3.416667,CCCCCC(=O)OCCCCC
4,0,0,15.961560,2.420953,4.841907,15.961560,1.140111,3.534439,3.290632,0.235045,...,45.774443,192.151415,5.651512,285,21,68.0,78.0,6.645833,3.180556,CC=CC(=O)C1C(C=CCC1(C)C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,0,0,18.169374,2.017464,4.034927,18.169374,1.211292,3.524117,3.356383,0.223759,...,43.650894,214.193280,5.224226,538,13,56.0,55.0,5.861111,3.916667,CCCCCCCCCCOC(=O)CC
2060,0,0,16.040654,2.628414,4.984756,16.040654,1.233896,3.532886,3.184509,0.244962,...,62.626381,182.130680,5.875183,215,22,74.0,94.0,5.597222,2.770833,CC1(C2CCC1(C(C2)OC=O)C)C
2061,0,0,14.445702,2.039694,4.079389,14.445702,1.203809,3.298990,3.056776,0.254731,...,39.535875,170.130680,5.671023,262,10,44.0,43.0,5.111111,3.166667,CCCCCCC(=O)OCC=C
2062,0,0,17.137765,2.563547,5.127093,17.137765,1.224126,3.597390,3.105208,0.221801,...,47.370948,194.130680,6.066584,305,18,76.0,91.0,5.756944,2.944444,CC(=O)OCC1=CCC2CC1C2(C)C


In [17]:
train_df_with_smiles.to_csv('train_mordred.csv', index=False)

In [19]:
calc = Calculator(descriptors)

results = []
test_smiles_list=test_df_optimal['canonical_SMILES'].tolist()
for smiles in test_smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    desc = calc(mol)
    desc_dict = desc.asdict()
    desc_dict['SMILES'] = smiles
    results.append(desc_dict)

# Создаем DataFrame
test_df_mordred = pd.DataFrame(results)

In [20]:
test_df_mordred

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,SMILES
0,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,14.537988,2.270653,4.541307,14.537988,1.211499,3.368485,...,41.913304,166.110613,6.388870,201,14,54.0,59.0,4.833333,2.861111,CC(C)CC1=NC=CN=C1OC
1,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,17.768236,2.350849,4.701699,17.768236,1.269160,3.564457,...,45.493017,185.120449,6.383464,308,18,70.0,79.0,4.444444,3.111111,CC(C)CC1=NC2=CC=CC=C2C=C1
2,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,40.567492,150.068080,7.146099,160,13,50.0,55.0,4.583333,2.611111,CC1=CC=C(C=C1)C(=O)OC
3,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,16.194221,2.160195,4.320390,16.194221,1.245709,3.436571,...,42.478296,178.099380,6.596273,304,12,56.0,58.0,4.472222,3.083333,CC(=O)OCCCC1=CC=CC=C1
4,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,11.542748,2.361624,4.723248,11.542748,1.154275,3.215783,...,40.316463,140.120115,5.389235,106,14,50.0,57.0,5.034722,2.152778,CC1CCCC(C1=O)(C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,11.945822,2.267184,4.534368,11.945822,1.194582,3.197666,...,39.310842,136.100048,6.186366,116,12,46.0,51.0,4.333333,2.361111,CCC1=NC=C(N=C1C)C
884,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,12.098438,2.208933,4.417866,12.098438,1.209844,3.188497,...,38.704239,138.079313,6.903966,124,10,44.0,47.0,3.722222,2.416667,CCOC1=NC(=CN=C1)C
885,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,28.409420,2.097472,4.194943,28.409420,1.183726,4.004715,...,55.607348,338.318481,5.126038,2062,23,96.0,96.0,9.333333,5.916667,CCCCCCCCCCCC(=O)OCCC(C)CCC=C(C)C
886,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,19.923254,2.194446,4.388892,19.923254,1.245203,3.635716,...,46.419841,218.130680,6.415608,563,16,68.0,71.0,5.222222,3.916667,CCCC=CCOC(=O)CC1=CC=CC=C1


In [21]:
# Удалить все нечисловые колонки кроме SMILES
non_numeric_cols = test_df_mordred.select_dtypes(exclude=[np.number]).columns
non_numeric_cols = [col for col in non_numeric_cols if col != 'SMILES']
test_df_with_smiles = test_df_mordred.drop(columns=non_numeric_cols)

In [22]:
test_df_with_smiles

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,SMILES
0,0,0,14.537988,2.270653,4.541307,14.537988,1.211499,3.368485,3.151377,0.262615,...,41.913304,166.110613,6.388870,201,14,54.0,59.0,4.833333,2.861111,CC(C)CC1=NC=CN=C1OC
1,0,0,17.768236,2.350849,4.701699,17.768236,1.269160,3.564457,3.408980,0.243499,...,45.493017,185.120449,6.383464,308,18,70.0,79.0,4.444444,3.111111,CC(C)CC1=NC2=CC=CC=C2C=C1
2,0,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,3.095000,0.281364,...,40.567492,150.068080,7.146099,160,13,50.0,55.0,4.583333,2.611111,CC1=CC=C(C=C1)C(=O)OC
3,0,0,16.194221,2.160195,4.320390,16.194221,1.245709,3.436571,3.156211,0.242785,...,42.478296,178.099380,6.596273,304,12,56.0,58.0,4.472222,3.083333,CC(=O)OCCCC1=CC=CC=C1
4,0,0,11.542748,2.361624,4.723248,11.542748,1.154275,3.215783,2.957574,0.295757,...,40.316463,140.120115,5.389235,106,14,50.0,57.0,5.034722,2.152778,CC1CCCC(C1=O)(C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,0,0,11.945822,2.267184,4.534368,11.945822,1.194582,3.197666,2.964744,0.296474,...,39.310842,136.100048,6.186366,116,12,46.0,51.0,4.333333,2.361111,CCC1=NC=C(N=C1C)C
884,0,0,12.098438,2.208933,4.417866,12.098438,1.209844,3.188497,2.942015,0.294202,...,38.704239,138.079313,6.903966,124,10,44.0,47.0,3.722222,2.416667,CCOC1=NC(=CN=C1)C
885,0,0,28.409420,2.097472,4.194943,28.409420,1.183726,4.004715,4.001640,0.166735,...,55.607348,338.318481,5.126038,2062,23,96.0,96.0,9.333333,5.916667,CCCCCCCCCCCC(=O)OCCC(C)CCC=C(C)C
886,0,0,19.923254,2.194446,4.388892,19.923254,1.245203,3.635716,3.341092,0.208818,...,46.419841,218.130680,6.415608,563,16,68.0,71.0,5.222222,3.916667,CCCC=CCOC(=O)CC1=CC=CC=C1


In [23]:
test_df_with_smiles.to_csv('test_mordered.csv', index=False)